# Conditions d'utilisation 
Pour que **InPoDa** fonctionne correctement vérifiez que les conditions suivantes doivent être réunies :
- Installation des modules : ```Pandas```, ```Textblob```, ```Spacy```, ```Translate```, ```Languedetect```, (```Gradio```?)
- Le fichier ***"aitweets.json"*** doit être dans le même dossier que **InPoDa**

# Supression des caractères spéciaux

In [66]:
# Etat : Fonctionne parfaitement 

def SuprCaracterSpe(Chaine):  # Prend en entrée une chaine de caractères 
    ChaineNettoyee = ''
    indice = 0
    while indice < len(Chaine):
        if (Chaine[indice] == 'R') and (indice < len(Chaine)-2) and (Chaine[indice] + Chaine[indice+1] == 'RT'): # On suprime les "RT"
            indice += 3
            while (indice < len(Chaine)-1) and Chaine[indice] != ' ':
                indice+=1
            ChaineNettoyee += ' '
            indice += 1
        elif (Chaine[indice].isalnum() == True):
            ChaineNettoyee += Chaine[indice]
            indice += 1
        elif (Chaine[indice] == "\ "[:-1]) and (indice < len(Chaine)-2) and (Chaine[indice] + Chaine[indice+1] == '\n'): # On suprime les sauts de ligne
            ChaineNettoyee += ' '
            indice += 1
        elif (Chaine[indice] == '@') and (indice < len(Chaine)-2):  # On suprime les pseudonymes (donc les arobases) 
            indice += 1
            while (indice < len(Chaine)-1) and Chaine[indice] != ' ':
                indice += 1
            ChaineNettoyee += ' '
            indice += 1
        else:
            ChaineNettoyee += ' '
            indice += 1
    return ChaineNettoyee     # Renvoie la chaine de cractères sans les caractères spéciaux

# Importation de nos propre modules

In [67]:
import tweet 
import inpoda

# Pour des raisons de clarté aucun abrégé ne sera utilisé pour faire appel a nos propre modules

# Récupération des Tweets du fichier Json → Json (Zone d'aterrissage)

In [68]:
# Etat : Fonctionne parfaitement

def ZoneAterrisage(NomFichier):
    from json import loads
    with open(NomFichier, 'r') as fs, open("Zone d’atterrissage.json", 'w') as fd:
        for ligne in fs:
            dicoL = loads(ligne)
            Tweet = dicoL["TweetText"]
            TweetN = SuprCaracterSpe(Tweet)  # Suprime tous les caractères spéciaux, les RT et les pseudonymes
            fd.write(f'{TweetN}\n')     # On écrit le Tweet nettoyé dans "Zone d’atterrissage.json"

fichier = "aitweets.json"
ZoneAterrisage(fichier)

# Récupération des Données du fichier Json → DataFrame ('DFP')

In [69]:
# Etat : Fonctionne parfaitement

def RecupDonneesP(NomFichier):  # Prend en entier le nom du fichier à analyser
    from pandas import DataFrame
    from pandas import concat
    from json import loads
    with open(NomFichier, 'r') as fs:
        ligne = fs.readline()
        dicoL = loads(ligne)
        dicoL["#"] = tweet.Hashtags(dicoL["TweetText"])
        dicoL["@"] = tweet.Arobases(dicoL["TweetText"])
        dicoL["Sentiment"] = tweet.Sentiment(dicoL["TweetText"])
        DFF = DataFrame([dicoL])   # On initialise le DatFrame qui va stocker en local les données de la base
        for ligne in fs:
            dicoL = loads(ligne)
            dicoL["#"] = tweet.Hashtags(dicoL["TweetText"])
            dicoL["@"] = tweet.Arobases(dicoL["TweetText"])
            dicoL["Sentiment"] = tweet.Sentiment(dicoL["TweetText"])
            DF2 = DataFrame([dicoL])
            DFF = concat([DFF,DF2])  
    DFF.index = list(i for i in range(0,1708))                
    return DFF

fichier = "aitweets.json"
DFP = RecupDonneesP(fichier)   # DFP correspond au DataFrame principal, dont les colonnes sont 'id','TweetText', etc
DFP

,id,AuthorLocation,CreatedAt,RetweetCount,TweetLanguage,TweetText,#,@,Sentiment
0,1415291904850153474,,2021-07-14T12:47:39Z,0,en,"@____bruvteresa_ According to research, NASA c...",[],[@____bruvteresa_],Positif
1,1415291947560828933,Mysore and BERLIN,2021-07-14T12:47:49Z,2,en,RT @HDataSystems: Artificial Intelligence and ...,"[#hdatasystems, #Artificia]",[@HDataSystems],Negatif
2,1415291877897605120,,2021-07-14T12:47:33Z,246,en,RT @adgpi: Army Technology Board conducted the...,[],[@adgpi],Negatif
3,1415291886860967940,,2021-07-14T12:47:35Z,1,en,"RT @pacorjo: According to a recent survey, the...",[],[@pacorjo],Positif
4,1415291968700264450,Internet,2021-07-14T12:47:54Z,20,en,RT @HarbRimah: Making AI Sing https://t.co/FJo...,"[#MachineLearning, #DataScience, #Python, #AI,...",[@HarbRimah],Neutre
...,...,...,...,...,...,...,...,...,...
1703,1421408743770791938,"Liverpool, England",2021-07-31T09:53:47Z,1,en,RT @innomaticshyd: Top 10 Real world Artificia...,[],[@innomaticshyd],Positif
1704,1421424066305605634,,2021-07-31T10:54:40Z,0,en,Iowa State part of U.S. National Science Found...,"[#ArtificialIntelligence, #IIoT, #GenerativeAd...",[],Positif
1705,1421423882427371521,127.0.0.1,2021-07-31T10:53:57Z,17,en,RT @intellimetri: Human Assisted #ArtificialIn...,"[#ArtificialIntelligence, #AI]","[@intellimetri, @nerdgirlz, @forbes, @ForbesBR...",Neutre
1706,1421423971858149377,"Singapore, Singapore",2021-07-31T10:54:18Z,12,en,RT @IainLJBrown: Artificial Intelligence learn...,[#Artificia],[@IainLJBrown],Positif


# Opérations d'Analyse
Méthode : Récupération des Tweets du **DataFrame** (pincipal) → **Analyse** → **DataFrame** (d'analyse)
Les Données d'Analyse seront liées aux données du **DataFrame** principal par leur index (le numéro de leur ligne tout simplement)

Puis pour finir : Fusion des différents **DataFrame** d'Analyse. 
Par soucis de complexité (noatmment de rapidité d'exécution) nous avons opté pour un regroupement de l'exécution des opérations d'Analyse de base (#,@,sentiments,topics) au sein d'une même fonction : ```DataFrameAnalyse()```

# Le Dictionnaire des Hashtags '#'

In [70]:
def DicoHashtags(DF):   # Entrée : DataFrame (la colonne des tweets)
    DicoH = {}
    from pandas import DataFrame
    Taille = DF.shape  # Tuple contenant (le nombre de ligne, le nombre de colonnes)
    for i in range(Taille[0]):
        ListeHashtags = DF.loc[i]
        for h in ListeHashtags:
            if h in DicoH:
                nombre = DicoH[h]
                nombre +=1
                DicoH[h] = nombre
            else:
                DicoH[h] = 1
    del DicoH['#']    # Il y a 33 # incomplets dans la base de données, ils sont pris en compte par la fonction, on prend donc le soin de suprimer cette clé du dico           
    return DicoH      # Renvoie un Dictionnaire contenant tous les Hashtags en clé et le nombre de fois qu'ils sont tweeté en valeur

DicoH = DicoHashtags(DFP["#"])

# Le Dictionnaire des Arobases @

In [71]:
def DicoArobases(DF):   # Entrée : DataFrame (la colonne des tweets)
    DicoA = {}
    from pandas import DataFrame
    Taille = DF.shape  # Tuple contenant (le nombre de ligne, le nombre de colonnes)
    for i in range(Taille[0]):
        ListeArobases = DF.loc[i]
        for h in ListeArobases:
            if h in DicoA:
                nombre = DicoA[h]
                nombre +=1
                DicoA[h] = nombre
            else:
                DicoA[h] = 1 
    del DicoA['@']               
    return DicoA      # Renvoie un Dictionnaire contenant tous les Hashtags en clé et le nombre de fois qu'ils sont tweeté en valeur

DicoA = DicoArobases(DFP["@"])

# Utilisateurs

L'enjeux est ici de compléter notre base de données incomplète. En effet notre base de données comporte pour chaque tweet un champ **ID** mais aucun "Pseudo" ou "Nom d'Utilisateur".
C'est pourquoi nous allons ajouter à notre **DataFrame** principal ce nouveau champ "Utilisateurs".

La création d'Utilisateurs est essentielle au bon déroulement de certaines opérations d'Analyses, notamment pour 'Les utilisateurs mentionnés par un utilisateur spécifique'. Il est donc impératif que tous les utilisateurs mentionnés dans les tweets soient présent au sein de la base de données, donc dans le champ "**Utilisateurs**" du DataFrame **DFP**.

In [72]:
# Etat : Fonctionne parfaitement

def CreationUtilisateurs(ListeD, nb, DFP):  # Entrée : Liste Utilisateurs mentionnés ('dict_keys'), nombre ('int'), DataFrame principal (DFP)
    from random import randint
    from pandas import DataFrame
    from pandas import concat
    ListeUtilisateurs = list(ListeD)
    ListeUtilisateurs.reverse()    # On inverse la liste des utilisateurs mentionnés pour éviter qu'un utilisateur se mentitonne lui même
    for i in range(nb):
        Lettre1 = chr(randint(65, 90))   # Intervalle Unicode des lettres Majuscules de l'alphabet latin
        Lettre2 = chr(randint(97, 122))  
        Lettre3 = chr(randint(97, 122))  # Intervalle Unicode des lettres Minuscules de l'alphabet latin
        Lettre4 = chr(randint(97, 122))
        Chiffre1 = chr(randint(48, 57))  # Intervalle Unicode des chiffres 
        Utilisateur = '@' + Lettre1 + Lettre2 + Lettre3 + Lettre4 + Chiffre1  # On ajoute '@' en début de Nom d'utilisateur pour s'assurer de l'homogéité des données.
        ListeUtilisateurs.append(Utilisateur)
    DFUtilisateurs = DataFrame({"Utilisateurs": ListeUtilisateurs})
    DFP = concat([DFUtilisateurs, DFP], axis=1)
    return DFP
   
DFP = CreationUtilisateurs(DicoA.keys(), (1708-len(DicoA)), DFP)
DFP

,Utilisateurs,id,AuthorLocation,CreatedAt,RetweetCount,TweetLanguage,TweetText,#,@,Sentiment
0,@TAMU,1415291904850153474,,2021-07-14T12:47:39Z,0,en,"@____bruvteresa_ According to research, NASA c...",[],[@____bruvteresa_],Positif
1,@inte,1415291947560828933,Mysore and BERLIN,2021-07-14T12:47:49Z,2,en,RT @HDataSystems: Artificial Intelligence and ...,"[#hdatasystems, #Artificia]",[@HDataSystems],Negatif
2,@ForbesBR,1415291877897605120,,2021-07-14T12:47:33Z,246,en,RT @adgpi: Army Technology Board conducted the...,[],[@adgpi],Negatif
3,@forbes,1415291886860967940,,2021-07-14T12:47:35Z,1,en,"RT @pacorjo: According to a recent survey, the...",[],[@pacorjo],Positif
4,@nerdgirlz,1415291968700264450,Internet,2021-07-14T12:47:54Z,20,en,RT @HarbRimah: Making AI Sing https://t.co/FJo...,"[#MachineLearning, #DataScience, #Python, #AI,...",[@HarbRimah],Neutre
...,...,...,...,...,...,...,...,...,...,...
1703,@Wixe3,1421408743770791938,"Liverpool, England",2021-07-31T09:53:47Z,1,en,RT @innomaticshyd: Top 10 Real world Artificia...,[],[@innomaticshyd],Positif
1704,@Oaoe9,1421424066305605634,,2021-07-31T10:54:40Z,0,en,Iowa State part of U.S. National Science Found...,"[#ArtificialIntelligence, #IIoT, #GenerativeAd...",[],Positif
1705,@Lexi3,1421423882427371521,127.0.0.1,2021-07-31T10:53:57Z,17,en,RT @intellimetri: Human Assisted #ArtificialIn...,"[#ArtificialIntelligence, #AI]","[@intellimetri, @nerdgirlz, @forbes, @ForbesBR...",Neutre
1706,@Ldjd9,1421423971858149377,"Singapore, Singapore",2021-07-31T10:54:18Z,12,en,RT @IainLJBrown: Artificial Intelligence learn...,[#Artificia],[@IainLJBrown],Positif


# Fonctionnalitées Suplémentaires :

- Actualisation manuelle des données 

In [73]:
# Etat : Fonctionne parfaitement

def Refresh():
    fichier = "aitweets.json"
    ZoneAterrisage(fichier)

Refresh()

# Les TOP-K 

- Top K hashtags

In [74]:
inpoda.TopKH(5, DicoH)

,#,Nombre de Tweets
11,#ArtificialIntelligence,207
5,#AI,204
2,#MachineLearning,86
27,#ai,52
3,#DataScience,50


- Top K Utilisateurs

In [75]:
inpoda.TopKU(5, DFP)

,Utilisateurs,RetweetCount
1194,@Qltk9,10327
1585,@Cbfb2,6481
1587,@Hifj5,6481
735,@LuxuryOpinions,1660
1047,@SecDef,1060


- Top K utilisateurs mentionnés

In [76]:
inpoda.TopKA(5, DicoA)

,@,Nombre de Mentions
26,@IainLJBrown,112
55,@Paula_Piccard,49
18,@nigewillson,24
679,@JoshuaBarbeau,17
977,@sankrant,17


# Le Nombre de publication par Utilisateur/Hashtag

- Le nombre de publications par utilisateur

In [77]:
inpoda.NbPostUtilisateur('@adgpi', DFP)

('@adgpi', 27)

- Le nombre de publications par hashtag

In [78]:
inpoda.NbPostHashtags("#AI", DicoH)

('#AI', 204)

# Les dernières analyses demandées

- L’ensemble de tweets d’un utilisateur spécifique

In [79]:
inpoda.TweetsUtilisateur("@AINewsFeed", DFP)

,Tweets postés par : @AINewsFeed
0,Artificial Intelligence System Market key tren...
1,Listen: An introduction to Artificial Intellig...
2,Major Study From Stanford University and Youpe...
3,"Science helps to cook the ""perfect steak""; Art..."
4,Artificial Intelligence To Police the World Wi...
5,Artificial intelligence wants to solve our ins...
6,"Artificial Intelligence in BFSI Market Trends,..."
7,Artificial intelligence helps improve NASA's e...
8,Open Access Articles - Artificial Intelligence...
9,RT @China4Tech: Futuristic hotel in China powe...


- L’ensemble de tweets mentionnant un utilisateur spécifique

In [80]:
inpoda.TweetsMentionUtilisateur("@intratio", DFP)

,Tweets mentionnant : @intratio
75,RT @intratio: $MDVL https://t.co/rso4WTkGcW Me...
400,RT @intratio: https://t.co/GGI5g1FRD3 Kinnate ...
843,RT @intratio: $CUE https://t.co/N3KHKWvbdp Cue...


- Les utilisateurs mentionnant un hashtag spécifique

In [81]:
inpoda.UtilisateurMentionHashtag('#AI', DFP)

,Utilisateurs
4,@nerdgirlz
8,@arXiv_Daily
10,@DecisionsSmart
42,@frontiersin
58,@TACHOUHONER
...,...
1679,@Fdpp9
1681,@Yhly9
1684,@Pydx4
1699,@Dmgg7


- Les utilisateurs mentionnés par un utilisateur spécifique

In [82]:
inpoda.UtilisateurMentionUtilsateur("@xoce_q", DFP)

,@xoce_q a mentionné :
0,@codewithibrahim


# Interface Graphique de Inpoda

- Gradio

# A faire !!!
- Régler le problème d'importation de d'autres modules au sein de mes propres modules
- Transformer la fonction 'RecupdonneP' pour que les données d'analyses soient dans DFP (puis suprimer DFA)
- Remplacer toutes les fonctions du script par des appels des deux modules 

# Ressources suplémentaires
Des ressources suplémentaire sont disponible sur le Github dont le lien est ci dessous. Vous y trouverez notamment un diagramme détaillant le fonctionnement de **InPoDa** ainsi que le détail de la répartition des taches. 

Github : https://github.com/LugolBis/Projet-IN304 
